In [26]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import os
import hashlib
import binascii

In [15]:
with open('aes/pw.txt', 'rb') as f:
    passphrase = f.read().strip()

In [121]:
with open('aes/data.txt', 'rb') as f:
    data = f.read()

In [122]:
len(data)

32

In [18]:
len(passphrase)

20

In [20]:
key = hashlib.sha256(passphrase).digest()[:16]
len(key)

16

In [133]:
%%time
iv = 0 # zero byte IV
cipher = Cipher(algorithms.AES(key), modes.CBC(iv.to_bytes(16, byteorder='little')), backend=default_backend())
encryptor = cipher.encryptor()
ct = encryptor.update(bytes(512))
#binascii.hexlify(ct)

CPU times: user 367 µs, sys: 22 µs, total: 389 µs
Wall time: 289 µs


In [28]:
%%time
decryptor = cipher.decryptor()

CPU times: user 149 µs, sys: 5 µs, total: 154 µs
Wall time: 110 µs


In [29]:
decryptor.update(ct)

b'123456789012345\nabcdefghijklmna\n'

In [319]:
ct = encryptor.update(data)
binascii.hexlify(ct)

b'01eaa3a7bbde99f559be0e7e0d4ff41137cf9cc0c6471d1ac393f00efe343952'

In [277]:
f = open('/dev/null', 'wb')
    

In [ ]:
f.write()

In [268]:
encryptor.finalize()

AlreadyFinalized: Context was already finalized.

In [136]:
with open('aespipe.txt', 'rb') as f:
    large = f.read()

In [169]:
class AESFile:
    def __init__(self, file, mode, passphrase):
        # actual file we are writing to
        # turn buffering off, as we are writing in chunks anyways
        self._file = open(file, mode=mode, buffering=0)
        # the encryption key is derived from the upper 16 bytes of the SHA256 hash (in case of AES128)
        self.key = hashlib.sha256(passphrase).digest()[:16]
        # aespipe in single-key mode uses a 0-byte IV which is incremented for each 512 byte sector 
        self.SECTOR_SIZE = 512 # bytes
        self.sector = 0
        self._cipher = self._get_cipher()
        self._encryptor = self._cipher.encryptor()
        self._decryptor = self._cipher.decryptor()
        self._encrypted_buffer = bytes(self.SECTOR_SIZE)
        
    def _next_sector(self):
        self.sector += 1
        self._cipher = self._get_cipher()
        self._encryptor = self._cipher.encryptor()
        self._decryptor = self._cipher.decryptor()
        
    def write(self, buffer):
        if len(buffer) != 512:
            raise NotImplementedError(f'Buffer Size has to be 512 bytes, not {len(buffer)}')
        # TODO: sector size
        #if len(buffer) > len(self._encrypted_buffer):
        #    self._encrypted_buffer = bytes(len(buffer))
        #written_bytes = 0
        #bytes_left = self._sector_size - self._sector_bytes 
        #print(f'we have {bytes_left} available in sector {self._sector}')
        #while written_bytes < len(buffer):
        #    write_slice = buffer[written_bytes:bytes_left]
        #    print(f'writing {len(write_slice)}')
        self._encrypted_buffer = self._encryptor.update(buffer)
        self._next_sector()
        return self._file.write(self._encrypted_buffer)

    
    def close(self):
        finalize = self._encryptor.finalize()
        self._file.close()
        if len(finalize):
            raise ValueError(f'Something went wrong with finalizing the encryptor, got length {len(finalize)}')
        # pad with zero bytes until the block (not sector!) is filled
        
    def _get_cipher(self):
        return Cipher(algorithms.AES(self.key), modes.CBC(self.sector.to_bytes(16, byteorder='little')), backend=default_backend())
        
class AESFile2:
    def __init__(self, file, mode, passphrase):
        # actual file we are writing to
        # turn buffering off, as we are writing in chunks anyways
        self._file = open(file, mode=mode, buffering=0)
        # the encryption key is derived from the upper 16 bytes of the SHA256 hash (in case of AES128)
        self.key = hashlib.sha256(passphrase).digest()[:16]
        # aespipe in single-key mode uses a 0-byte IV which is incremented for each 512 byte sector 
        self.SECTOR_SIZE = 512 # bytes
        self.sector = 0
        self._cipher = self._get_cipher()
        self._encrypted_buffer = bytes(self.SECTOR_SIZE)
        
    def _next_sector(self):
        self.sector += 1
        self._cipher = self._get_cipher()
        
    def write(self, buffer):
        if len(buffer) != 512:
            raise NotImplementedError(f'Buffer Size has to be 512 bytes, not {len(buffer)}')
        # TODO: sector size
        #if len(buffer) > len(self._encrypted_buffer):
        #    self._encrypted_buffer = bytes(len(buffer))
        #written_bytes = 0
        #bytes_left = self._sector_size - self._sector_bytes 
        #print(f'we have {bytes_left} available in sector {self._sector}')
        #while written_bytes < len(buffer):
        #    write_slice = buffer[written_bytes:bytes_left]
        #    print(f'writing {len(write_slice)}')
        self._encrypted_buffer = self._cipher.encrypt(buffer)
        self._next_sector()
        return self._file.write(self._encrypted_buffer)
        
    def close(self):
        self._file.close()
        # pad with zero bytes until the block (not sector!) is filled
        
    def _get_cipher(self):
        return AES.new(self.key, AES.MODE_CBC, IV=self.sector.to_bytes(16, byteorder='little'))

In [166]:
aes_file = AESFile2('test.aes', 'wb', passphrase)

In [167]:
aes_file.write(huge[:1024])

1024

In [86]:
with open('aespipe.txt', 'rb') as f:
    data = f.read()

In [134]:
import subprocess

In [135]:
data

b'123456789012345\nabcdefghijklmna\n'

In [140]:
%%time
data
p = subprocess.Popen(['aespipe', '-P', 'pw.txt'], stdout=subprocess.PIPE, stdin=subprocess.PIPE)
result = p.communicate(input=bytes(128*1024))

CPU times: user 521 µs, sys: 4.04 ms, total: 4.56 ms
Wall time: 4.95 ms


In [87]:
6144/512

12.0

In [76]:
len(data)

6532

In [92]:
for i in range(len(data) // 512):
    aes_file.write(data[512*i:512*(i+1)])

In [84]:
aes_file.close()

In [61]:
import tarfile

In [179]:
aesfile = AESFile2('test.tar.aes', 'wb', passphrase)
normalfile = open('test.normal.tar', 'wb', buffering=0)
archive = tarfile.open(fileobj=aesfile, mode='w|', bufsize=512)

In [180]:
archive.add('aespipe.txt')

In [177]:
archive.close()

In [181]:
archive.add('aespipe.ipynb')

In [185]:
aesfile.sector

484462

In [184]:
%prun archive.add('/home/sk/Downloads/yolov3.weights')

         7841187 function calls in 4.485 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   484390    1.065    0.000    1.065    0.000 {method 'encrypt' of '_AES' objects}
   484390    0.655    0.000    0.655    0.000 {method 'write' of '_io.FileIO' objects}
    15140    0.467    0.000    4.425    0.000 tarfile.py:443(__write)
   484390    0.387    0.000    3.928    0.000 <ipython-input-169-e3c446c0b53e>:66(write)
   484390    0.305    0.000    0.737    0.000 blockalgo.py:136(__init__)
   484390    0.254    0.000    1.466    0.000 <ipython-input-169-e3c446c0b53e>:86(_get_cipher)
   484390    0.227    0.000    0.227    0.000 {built-in method Crypto.Cipher._AES.new}
   484390    0.206    0.000    0.942    0.000 AES.py:55(__init__)
   484390    0.201    0.000    1.143    0.000 AES.py:61(new)
   484390    0.187    0.000    1.653    0.000 <ipython-input-169-e3c446c0b53e>:62(_next_sector)
   484390    0.147    0.000    0.204    0

In [73]:
aesfile.sector

20

In [ ]:
to_write = 512
left = 400


In [333]:
b'12345'[:512]

b'12345'

In [360]:
!cat 'test.aes'

�}_��60\ ~J��[TU���U'�����A���6�

In [373]:
dir(encryptor)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_ctx',
 'finalize',
 'update',
 'update_into']

In [141]:
with open('/home/sk/Downloads/yolov3.weights', 'rb') as f:
    huge = f.read()

In [144]:
%%time
aesfile = AESFile('huge.aes', 'wb', passphrase)
for i in range(len(huge) // 512):
    aesfile.write(huge[512*i:512*(i+1)])

CPU times: user 13.1 s, sys: 708 ms, total: 13.8 s
Wall time: 13.9 s


In [170]:
%%time
aesfile = AESFile2('huge.aes', 'wb', passphrase)
for i in range(len(huge) // 512):
    aesfile.write(huge[512*i:512*(i+1)])

CPU times: user 2.73 s, sys: 448 ms, total: 3.17 s
Wall time: 3.17 s


In [145]:
from Crypto.Cipher import AES

In [148]:
cipher = AES.new(key, AES.MODE_CBC, IV=bytes(16))

In [149]:
cipher

In [151]:
ct_bytes = cipher.encrypt(huge[:512])

In [152]:
ct_bytes

b'\x9c\xcb\x88/\x94k\xe9\xcf\x9b\xe2e%\xd6uU\x18\xde\xbd\x17\xb2^(\x9fO\xf6\x80WY\xae\xcdsw\xb7\xd1\x0e\x0c\xfb\x9cf\x1eJ)\x80\xfe\x80\xd4\x04\xa0"Lg\x86\xec\xae\xed\xab \xc8t\xc8\xb1\xe1\x8e7\x89\xbd\xfa4\x92\x94*l\xf64Z\xc4\xabt2\xb6\x89M\xb3\x94\xae*_\x8eF>\xac0\xcf\xfe\xf4w>\xa0\x0e\x022\x0b\x8f\xc5\x9e\xe8\xe2\xe2&\x168\x15\rpCr\xc2E1\x1c\xca\x1c\xcc\x12F\xc4\xb5\xcd\xfe"\xed\xef\xb2\xaeW\x17\xed\x82\x19\x1f\xd6D\x196f\x0e\xc8\xad\x9d8\xe6\xb5]\x8aK\x0c\xf9\x9c\xe7]\xa8\xbf\xbe\xf6\xde\xd3\xeb\x8bn\t\xdcl\xe4\xb7\x85HZ\x92N\x9f\xbd\x17\xfb\xc2\n\xbe\xdf\xa3D\x0b8>\xd1\xb3u\xda\xf4\x8b\xa3h\x89\xb5\xa6=<\x8a\xd6\xa9\x11\xca\xa4\x05\xe6\xae\x05WMvjU.1A.\xae|\xb4j\x8bV\xc2\x97\x08?nH\xca\xa6w\x9a\x96Z\xea\xf3\x1a%\xf2\xe6_\xc6\x0f\xab\xe2\x92{\xe1i\xb1\r#\x98\xec\xbdm\x07$\x98r0"\x88\xb3p\x94\x95\xd0\x81T5y\xac\xe2\xdd\xed\xe4\x01\xb2\xa0^%W\x1e-$\xf0\xfd\x89\xbd+)\xe7\x14\xdd\xa6\xbc\xfd0\xb3\xf2A\x84\xd0AE\xa5\x08\x1e\x9d\x120\xf9\x04\xc14U\xfe\x1e\xda\xd5o\xe5\x12\xd0\xfc\xf6\x13\

In [154]:
cipher.IV = bytes(16)

In [156]:
dir(cipher)

['IV',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cipher',
 'block_size',
 'decrypt',
 'encrypt',
 'mode']